In [ ]:
import requests
import json
from slugify import slugify
import pandas as pd
import datetime as dt
from _polyline_decoder import decode_polyline

endpoint = "https://maps.googleapis.com/maps/api/directions/json"
key = ""

 - driving
 - walking

In [ ]:
trips = pd.read_csv("_trips.csv", parse_dates=['date'])
trips.head()

In [ ]:
trips.info()

In [ ]:
def make_request(origin, destination, mode):
    params = {
        'origin': "%f,%f" % origin,
        'destination': "%f,%f" % destination,
        'mode':mode,
        'key': key
    }
    res=requests.get(endpoint, params=params)
    return json.loads(res.text)

def get_polylist(result):
    points_ = decode_polyline(result["routes"][0]["overview_polyline"]["points"])
    return [[lat, long] for lat, long in points_]

In [ ]:
def make_request_from_row(row):
    result = make_request(origin=(row["origin_lat"],row["origin_long"]),
                          destination=(row["destination_lat"],row["destination_long"]), 
                          mode=row["mode"])
    return result

trips["result"]=trips.apply(make_request_from_row, axis=1)
trips.head()

In [ ]:
trips["polyline"] = trips["result"].apply(get_polylist)
trips.head()

In [ ]:
trips["id"] = trips["name"].apply(lambda s: slugify(s, separator='_'))
trips.head()

In [ ]:
color_mapping = {
    'hitchhiking': 'red',
    'walking': 'blue',
    'bus':'yellow'
}

def get_info(r):
    info = {
#        'id': r['id'],
        'name': r['name'],
        'date': dt.datetime.strftime(r['date'], '%Y-%m-%dT%H:%M:%SZ'),
        'description': None if pd.isna(r['description']) else r['description'],
        'color': color_mapping[r['our_mode']],
        'points': r["polyline"]
    }
    routes = r['result']['routes']
    
    if len(routes) > 0:
        route = routes[0]
        info['bounds'] = route["bounds"]
        if len(route["legs"]) > 1:
            print(r['name'], "has more than one leg")
        leg = route["legs"][0]
        info["origin"] = leg["start_address"]
        info["destination"] = leg["end_address"]
        info["distance"] = leg["distance"]
    return info

trips_expanded = trips.apply(get_info, axis=1)

In [ ]:
routes = dict(zip(trips.id.values,trips_expanded.values))